<a href="https://colab.research.google.com/github/charliewei0716/demo-azure-cognitive-search-python-vector-search/blob/main/demo-azure-cognitive-search-python-vector-search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install azure-search-documents==11.4.0b11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.5/312.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.4/192.4 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential

search_index_client = SearchIndexClient(
    endpoint="https://charlie-test.search.windows.net",
    credential=AzureKeyCredential("YOUR-SEARCH-API-KEY") # 你的 Cognitive Search 金鑰
)

In [3]:
from azure.search.documents.indexes.models import SimpleField, SearchableField, SearchFieldDataType, SearchField
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True, sortable=True, filterable=True, facetable=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    SearchableField(name="category", type=SearchFieldDataType.String, filterable=True),
    SearchField(name="titleVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1536, vector_search_profile="HnswProfile"),
    SearchField(name="contentVector", type=SearchFieldDataType.Collection(SearchFieldDataType.Single), vector_search_dimensions=1536, vector_search_profile="HnswProfile"),
]

In [4]:
from azure.search.documents.indexes.models import (
    HnswVectorSearchAlgorithmConfiguration, VectorSearchAlgorithmKind, HnswParameters, ExhaustiveKnnVectorSearchAlgorithmConfiguration, ExhaustiveKnnParameters
)
algorithms=[
    HnswVectorSearchAlgorithmConfiguration(
        name="Hnsw",
        kind=VectorSearchAlgorithmKind.HNSW,
        parameters=HnswParameters(
            m=4,
            ef_construction=400,
            ef_search=500,
            metric="cosine",
        )
    ),
    ExhaustiveKnnVectorSearchAlgorithmConfiguration(
        name="ExhaustiveKnn",
        kind=VectorSearchAlgorithmKind.EXHAUSTIVE_KNN,
        parameters=ExhaustiveKnnParameters(
            metric="cosine",
        )
    )
]

In [5]:
from azure.search.documents.indexes.models import AzureOpenAIVectorizer, AzureOpenAIParameters
vectorizers=[
    AzureOpenAIVectorizer(
        name="AzureOpenAI",
        kind="azureOpenAI",
        azure_open_ai_parameters=AzureOpenAIParameters(
            resource_uri="https://charlie-oai.openai.azure.com/",
            deployment_id="text-embedding-ada-002",
            api_key="OPENAI_API_KEY" # 你的 Azure OpenAI 金鑰
        )
    )
]

In [6]:
from azure.search.documents.indexes.models import VectorSearchProfile
profiles=[
    VectorSearchProfile(
        name="HnswProfile",
        algorithm="Hnsw",
        vectorizer="AzureOpenAI"
    ),
    VectorSearchProfile(
        name="ExhaustiveKnnProfile",
        algorithm="ExhaustiveKnn",
        vectorizer="AzureOpenAI"
    )
]

In [7]:
from azure.search.documents.indexes.models import VectorSearch, SearchIndex

vector_search = VectorSearch(profiles=profiles, algorithms=algorithms, vectorizers=vectorizers)
index = SearchIndex(name="text-sample-vertor", fields=fields, vector_search=vector_search)
result = search_index_client.create_or_update_index(index)
print(f' {result.name} created')

 text-sample-vertor created


In [10]:
import json
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

with open('docVectors.json', 'r') as file:
    documents = json.load(file)

search_client = SearchClient(
    endpoint="https://charlie-test.search.windows.net",
    index_name="text-sample-vertor",
    credential=AzureKeyCredential("YOUR-SEARCH-API-KEY") # 你的 Cognitive Search 金鑰
)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents")

Uploaded 108 documents


In [11]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

search_client = SearchClient(
    endpoint="https://charlie-test.search.windows.net",
    index_name="text-sample-vertor",
    credential=AzureKeyCredential("YOUR-SEARCH-API-KEY") # 你的 Cognitive Search 金鑰
)

In [12]:
from azure.search.documents.models import VectorizableTextQuery

query = "tools for software development"
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.82971567
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.81866795
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, suc

In [14]:
from azure.search.documents.models import VectorizableTextQuery

query = "軟體開發工具"
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.7880139
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure App Service
Score: 0.78773326
Content: Azure App Service is a fully managed platform for building, deploying, and scaling web apps. You can host web apps, mobile app backends, and RESTful APIs. It supports a variety of programming languages and frameworks, such as .NET, Java, Node.js, Python, and PHP. The service of

In [16]:
from azure.search.documents.models import VectorizableTextQuery

query = "tools for software development"
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector", exhaustive=True)

results = search_client.search(
    search_text=None,
    vector_queries= [vector_query],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.8297154
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8186685
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

In [18]:
from azure.search.documents.models import VectorizableTextQuery

query = "tools for software development"
vector_query = VectorizableTextQuery(text=query, k=3, fields="titleVector, contentVector")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["title", "content", "category"]
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var

In [19]:
from azure.search.documents.models import VectorizableTextQuery

query = "tools for software development"
vector_query_1 = VectorizableTextQuery(text=query, k=3, fields="titleVector")
vector_query_2 = VectorizableTextQuery(text=query, k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query_1, vector_query_2],
    select=["title", "content", "category"],
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var

In [20]:
from azure.search.documents.models import VectorizableTextQuery
from azure.search.documents.models import VectorFilterMode

query = "tools for software development"
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    vector_filter_mode=VectorFilterMode.PRE_FILTER,
    filter="category eq 'Developer Tools'",
    select=["title", "content", "category"],
    top=3
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.8297154
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.8186685
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports various platforms, such 

In [22]:
from azure.search.documents.models import VectorizableTextQuery

query = "tools for software development"
vector_query = VectorizableTextQuery(text=query, k=3, fields="contentVector")

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["title", "content", "category"],
    top=3
)

for result in results:
    print(f"Title: {result['title']}")
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['content']}")
    print(f"Category: {result['category']}\n")

Title: Azure DevOps
Score: 0.03333333507180214
Content: Azure DevOps is a suite of services that help you plan, build, and deploy applications. It includes Azure Boards for work item tracking, Azure Repos for source code management, Azure Pipelines for continuous integration and continuous deployment, Azure Test Plans for manual and automated testing, and Azure Artifacts for package management. DevOps supports a wide range of programming languages, frameworks, and platforms, making it easy to integrate with your existing development tools and processes. It also integrates with other Azure services, such as Azure App Service and Azure Functions.
Category: Developer Tools

Title: Azure DevTest Labs
Score: 0.032786883413791656
Content: Azure DevTest Labs is a fully managed service that enables you to create, manage, and share development and test environments in Azure. It provides features like custom templates, cost management, and integration with Azure DevOps. DevTest Labs supports var